# Project 3: Munging and analyzing data from the web

In [ ]:
from bs4 import BeautifulSoup
import requests
import seaborn as sns
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36'}

html_best = requests.get('https://www.imdb.com/chart/top/?ref_=nv_mv_250', headers=headers)
html_worst = requests.get('https://m.imdb.com/chart/bottom/', headers=headers)
soup_best = BeautifulSoup(html_best.text)
soup_worst = BeautifulSoup(html_worst.text)
display(soup_worst)

In [ ]:
top_movies = soup_best.find_all('li', attrs={'class': 'ipc-metadata-list-summary-item sc-1364e729-0 caNpAE cli-parent'})

titles_best = [re.sub(r'\d+\.\s*','' ,str(item.find('h3', attrs={'class':'ipc-title__text'}).contents[0])) for item in top_movies]
display(titles_best)

bottom_movies = soup_worst.find_all('li', attrs={'class': 'ipc-metadata-list-summary-item sc-1364e729-0 caNpAE cli-parent'})

titles_worst = [re.sub(r'\d+\.\s*','' ,str(item.find('h3', attrs={'class':'ipc-title__text'}).contents[0])) for item in bottom_movies]
display(titles_worst)

In [ ]:
ratings = soup_best.find_all('div', attrs={'class': 'sc-be6f1408-7 iUtHEN cli-title-metadata'})
rating_items = [item.find_all('span', attrs={'class': 'sc-be6f1408-8 fcCUPU cli-title-metadata-item'}) for item in ratings]
display(rating_items)

In [ ]:
df = pd.DataFrame()

years_list_best = [] 
length_list_best = []
rating_list_best = []

for i in rating_items:

    
    years_list_best.append(int(i[0].contents[0]))
    length_list_best.append(i[1].contents[0])
    if len(i) > 2:
        rating_list_best.append(i[2].contents[0])
    else:
        rating_list_best.append('Not Rated')
df['Years'] = years_list_best
df['Length'] = length_list_best
df['Rating'] = rating_list_best
df
